## Candidate Embedding 구하고 VectorDB에 인덱싱하기

### 1. candidate_model 가지고 item feature 를 candidate embedding으로 변환하기

#### 1-1. candidate_model 로드

In [1]:
import boto3

AWS_PROFILE = 'default'
BUCKET_NAME = 'jimin-model'
session = boto3.Session(profile_name=AWS_PROFILE)
s3_client = session.client('s3')

In [2]:
from utils.load_model import LoadModel
import tensorflow as tf

load_model = LoadModel(s3_client, BUCKET_NAME)
candidate_model = load_model.call("candidate_model")

2024-05-03 12:09:55.914972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### 1-2. feature 테이블에서 가져온 데이터 가공해서 item dateset 준비

In [3]:
import mysql.connector as sql

import pandas as pd

host = 'ssm-develop.db.sinsang.market'
port = 3306
username = 'dealicious'
password = 'tlstkd12!@'
database_name = 'dealicious'

db_connection = sql.connect(host=host, database=database_name, user=username, password=password)

retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)

retrievals_df.head(3)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_61920/1062528239.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)


,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided


In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(retrievals_df, test_size=0.2, random_state=42)

In [5]:
train_df["article_id"] = train_df["article_id"].astype(str)
test_df["article_id"] = test_df["article_id"].astype(str)

In [6]:
input_signature = candidate_model.signatures["serving_default"].structured_input_signature[-1]
candidate_features = list(input_signature.keys())

# Get list of unique candidate items.
item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)

item_ds = tf.data.Dataset.from_tensor_slices(
    {col: item_df[col] for col in item_df})


/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_61920/753176361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_df.drop_duplicates(subset="article_id", inplace=True)


#### 1-3. candidate embedding 계산하기

In [7]:
# Compute embeddings for all candidate items.
candidate_embeddings = item_ds.batch(448).map(
    lambda x: (x["article_id"], candidate_model(x)))

### 2. vector db에 candidate embedding 인덱싱하기

In [8]:
import requests
import json
import weaviate

# weaviate schdema name
INDEX_NAME = 'ItemCandidate'
weaviate_client = weaviate.Client("https://weaviate.dev.sinsang.market")

/Users/jimin/study/two-tower/.venv/lib/python3.9/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.7.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [25]:
# 스키마 목록
response = weaviate_client.schema.get()
classes = [c['class'] for c in response['classes']]
print(classes)

weaviate_client.query.aggregate('ItemCandidate').with_meta_count().do()

['Article', 'Dealidocsall5', 'ItemCandidate']


{'data': {'Aggregate': {'ItemCandidate': [{'meta': {'count': 6487}}]}}}

In [10]:
# batch import embeddings
def bulk_insert(item_id_list, embedding_list):
    weaviate_client.batch.configure(batch_size=100)
    with weaviate_client.batch as batch:
        for item_id, embedding in zip(item_id_list, embedding_list):
            properties = {
                "item_id": int(item_id)
            }
            batch.add_data_object(properties, INDEX_NAME, vector=embedding.tolist())
  

In [24]:
for batch in candidate_embeddings:
    item_id_list, embedding_list = batch
    item_id_list = item_id_list.numpy().astype(int)
    embedding_list = embedding_list.numpy()

    bulk_insert(item_id_list, embedding_list)

### 3. 샘플 쿼리

In [18]:
import numpy as np

emb_dim = 16
embedding = np.random.rand(emb_dim)

In [32]:
k = 10
nearVector = {
    "vector": embedding
}
result = weaviate_client.query.get(
        INDEX_NAME, ["item_id"]
    ).with_near_vector(
        nearVector
    ).with_limit(k).with_additional("vector").do()


print(json.dumps(result, indent=4))

{
    "data": {
        "Get": {
            "ItemCandidate": [
                {
                    "_additional": {
                        "vector": [
                            -0.023482554,
                            -0.37243342,
                            0.25602636,
                            -0.37349474,
                            0.05387725,
                            0.21127278,
                            0.08824536,
                            0.3604427,
                            0.9581325,
                            0.0064371377,
                            -0.07189374,
                            0.14129508,
                            -0.41921556,
                            -0.37611598,
                            0.2706183,
                            -0.78729665
                        ]
                    },
                    "item_id": 617279006
                },
                {
                    "_additional": {
                        "vector": [

In [22]:
# 스키마 삭제
# weaviate_client.schema.delete_class(INDEX_NAME)